In [1]:
# import pyodbc
import pandas as pd
from datetime import date
from datetime import timedelta
from openpyxl import load_workbook
import openpyxl
import win32com.client as win32
import os
import glob2
from pathlib import Path

In [20]:
# closes all excel files
def closeFile():
    try:
        os.system('TASKKILL /F /IM excel.exe')

    except Exception:
        print("KU")

closeFile()

### update Sharepoint lists

In [23]:
path1 = r'C:\Users\GisselleMaira\Desktop\Sharepoint\OpportunityList2.xlsx'
path2 = r'C:\Users\GisselleMaira\Desktop\Sharepoint\ClosedOpps.xlsx'

# directory = 'C:\\Users\\GisselleMaira\\Desktop\\Sharepoint'
# files  = os.listdir(directory)[0:2]

In [22]:
xla = win32.DispatchEx("Excel.Application")
xla.CalculateUntilAsyncQueriesDone()
xla.DisplayAlerts = False
xla.Visible = True
wb1=xla.Workbooks.Open(Filename=path1)
wb1.RefreshAll()
wb1.Save()
wb1.Close()
xla.Quit()

# xlapp.Quit()
# wbCloseA = xla.Workbooks.Close()
# wb.Close(True, r'C:\Path\to\folder\Test.xlsx')

In [24]:
xl = win32.DispatchEx("Excel.Application")
xl.CalculateUntilAsyncQueriesDone()
xl.DisplayAlerts = False
xl.Visible = True
wb2= xl.Workbooks.Open(Filename=path2)
wb2.RefreshAll()
wb2.Save()
wb2.Close()
xl.Quit()

In [27]:
# read refreshed data
opps = pd.read_excel(path1)
closed = pd.read_excel(path2)


In [37]:
# closed_hist = closed[closed['Active?'] == False].drop(['ID', 'Item Type', 'Path'],1)
# closed_hist.to_excel(r'C:\Users\GisselleMaira\Desktop\closed_hist.xlsx', index = None, header=True)

In [112]:
date.today()  + timedelta(days=-1)

datetime.date(2022, 1, 18)

In [113]:
history1 = pd.read_excel( r'C:\Users\GisselleMaira\Desktop\Opps_History.xlsx')
history2 = pd.read_excel( r'C:\Users\GisselleMaira\Desktop\closed_hist.xlsx')
# history2['saved on'] = date.today()  + timedelta(days=-1)

In [80]:
# see what's changed for active
df_left_active = opps.merge(history1.drop_duplicates(), on=['Client Company', 'Opportunity/Project Name',\
                                                            'Notes', 'Modified', 'Win Likelihood','On hold'], 
                                how='left', indicator=True)

df_left_active['_merge'].value_counts()

both          54
right_only     0
left_only      0
Name: _merge, dtype: int64

In [82]:
# changed_active = df_left_active[df_left_active['_merge'] == 'left_only']
# changed_items_active = opps.iloc[changed_active.index, :]
# all_active = pd.concat([history1, changed_items_active], axis=0)

In [114]:
# see what's changed for closed
df_left_closed = closed.merge(history2.drop_duplicates(), on=['Client Company', 'Opportunity/Project Name',\
                                                            'Notes', 'Modified'], how='left', indicator=True)

df_left_closed['_merge'].value_counts()

both          281
left_only      54
right_only      0
Name: _merge, dtype: int64

In [109]:
changed_closed = df_left_closed[df_left_closed['_merge'] == 'left_only']
closed_edit = closed.drop(['ID','Item Type', 'Path'], 1)
changed_items_closed = closed_edit.iloc[changed_closed.index, :]


In [115]:
changed_items_closed['saved on'] = date.today() 
all_closed = pd.concat([history2, changed_items_closed], axis=0)

<ipython-input-115-a76659857223>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  changed_items_closed['saved on'] = date.today()


In [ ]:
all_closed.to_excel(r'C:\Users\GisselleMaira\Desktop\closed_hist.xlsx')
all_active.to_excel(r'C:\Users\GisselleMaira\Desktop\Opps_History.xlsx')



In [ ]:
Active = opps['Active?'].count()
today = date.today()
now = datetime.now()
last_sunday  = today + timedelta(days=-today.weekday()-1, weeks=0)



In [ ]:
opps = opps.sort_values(by ='Created', ascending=False)

new = opps[opps['Created'] >= pd.Timestamp(last_monday )].shape[0]

LatestDate = opps['Created'].max()

In [ ]:
outlook = win32.Dispatch('outlook.application')
    
mail = outlook.CreateItem(0)
# mail.To = 'paul.blodgett@spinnakerconsultinggroup.com'
mail.cc = 'gisselle.maira@spinnakerconsultinggroup.com'
mail.Subject = 'Opportunity update'
    
mail.Body = f"""Last week (wk ending {last_sunday}) there were {new} new opportunties and {Active} total opportunities.
    
The last item created was entered at {LatestDate}.  Sent {now}."""
    
mail.Send()   

In [ ]:
# book = load_workbook(path)
# writer = pd.ExcelWriter(path, engine = 'openpyxl')
# writer.book = book

# all_data.to_excel(writer, sheet_name = 'x3')

# writer.save()
# writer.close()

In [ ]:
#     query_string = """SELECT * 
# FROM sec_edgar_searchqueryresult sqr
# JOIN sec_edgar_searchqueryterm sqt ON sqt.id = sqr.term_id
# JOIN sec_edgar_filingdocument fd ON fd.id = sqr.filing_document_id
# JOIN sec_edgar_filing f ON f.id = fd.filing_id
# JOIN sec_edgar_companyinfo ci ON ci.company_id = f.company_id AND ci.date = f.date_filed 
# WHERE sqr.search_query_id = {0}
# ORDER BY f.date_filed, f.company_id
# """.format(search_query_id)
#     query_df = pandas.read_sql(query_string, conn)

## CREATE NEW TABLE

In [ ]:
# CREATE NEW TABLE
# cursor.execute('create table new_table_name (
#                                              column1 datatype,
#                                              column2 datatype,
#                                              column3 datatype,
#                                              ....
#                                              )')
# cursor.commit()

In [ ]:
# cursor.execute('select * into new_table_name'
#                'from [Excel 12.0 Xml;HDR=YES;IMEX=2;ACCDB=YES;DATABASE=E:\PythonInOffice\python_connect_to_ms_access\richest10.xlsx].[Sheet1$]')

# cursor.commit()

In [ ]:
# cursor.execute('select * into new_table_name'
#                'from [Excel 12.0 Xml;HDR=YES;IMEX=2;ACCDB=YES;DATABASE=E:\PythonInOffice\python_connect_to_ms_access\richest10.xlsx].[Sheet1$]')

# cursor.commit()

# # #######################ALT EXCEL CONNECTION WITH AUTOCOMMIT############################
# conn1 = pyodbc.connect('DRIVER={Microsoft Excel Driver \ 
#                        (*.xls)};DBQ='+exFile,autocommit=True)

# # APPEND TO LOCAL TABLE
# sql = """INSERT INTO ETABS
#          SELECT * FROM [Excel 12.0 Xml; HDR = Yes;Database={myfile}].[Sheet1$];
#       """

# curs = conn.cursor()
# curs.execute(sql.format(myfile = exFile))
# conn.commit()

## DELETE OPTIONS

In [ ]:
# checks for driver
# [i for i in pyodbc.drivers() if i.startswith('Microsoft Access Driver')]
# ['Microsoft Access Driver (*.mdb, *.accdb)'] 64-bit
# Microsoft Access Driver (*.mdb) 32-bit

# DBFile = r'C:\Users\GisselleMaira\Documents\SLT_data.accdb'
# exFile = r'C:\Users\GisselleMaira\Desktop\OpportunityList2.xlsx'

# connect to Access
# conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+DBFile)
# cursor = conn.cursor()

# get available tables
# cursor = conn.cursor()
# for i in cursor.tables(tableType='TABLE'):
#     print(i.table_name)

    # for i in cursor.tables(tableType='VIEW'):
#     print(i.table_name)

#Run SQL statement
# cursor.execute('select * from Opps_All')
# rows = cursor.fetchall()
# #get one row
# one_row = cursor.fetchone()

# df = pd.read_sql('select * from Opps_Active', conn)
# df = df.drop(['ID1', 'Path', 'Date Added'], 1)

In [ ]:
# # #######################   DELETE TABLE ############################
# cursor.execute('DROP TABLE table_name')
# cursor.commit()

# # #######################   DELETE ROWS ############################
# cursor.execute('DELETE FROM table_name'
#                'WHERE condition')
# cursor.commit()

## INSERT/UPDATE

In [ ]:
# cursor.execute('INSERT INTO table_name (column1, column2, column3 ...)'
#                'VALUES ('value1', 'value2', 'value3' ...)')
# cursor.commit()

# cursor.execute('UPDATE table_name'
#                'SET column1 = value1, column2 = value2, ...'
#                'WHERE condition')
# cursor.commit()

# curs.executemany('insert into mytable (mycolumn) values (?)', excel_results)
# conn.commit()

# # get column names
# for row in curs1.tables():
#     print row.table_name